# AVIRIS‑NG Model Testing

Hyperspectral imagery from the AVIRIS‑NG instrument captures reflectance across hundreds of contiguous spectral bands, providing an opportunity to differenciate between species based on their spectral signatures. In this notebook, I applied some statistical and machine‑learning techniques to detect *Leucadendron argenteum* (the silvertree) within AVIRIS-NG scenes acquired during the BioSCape campaign. I first parsed metadata from the ENVI header, load occurrence points and reproject them to the raster's coordinate system, sample spectra at both positive (silvertree) and background locations, and then trained a classifier using gradient boosting (BioSCape model testing tutorial). Performance was evaluated on a hold‑out test set, and results inform us about subsequent refinements such as band selection and anomaly detection. In the second part of the notebook (ChatGPT method) I used an alternative detector based on a matched‑filter approach and examined performance across multiple scenes.

## Defining file paths

To organise the workflow, we start by specifying paths to the key data sources. The `gpkg_path` points to a geopackage containing occurrence records of *L. argenteum*, and `cube_path` references a Level 2A AVIRIS reflectance data cube. The header path is derived by appending `.hdr` to the cube path.

In [49]:
# Set paths
gpkg_path   = "data/silvertree/occurrence.gpkg"                                     # GBIF/iNat geopackage
cube_path   = "data/AVIRIS_L3/ang20231126t084422_002_L2A_OE_main_27577724_RFL_ORT"  # data file (no .hdr)
hdr_path    = cube_path + ".hdr"                                                    # companion header

## Importing libraries

The analysis relies on a combination of geospatial and numerical libraries. `geopandas` and `shapely` handle vector data and spatial operations; `xarray` and `rasterio` manage raster data; `numpy` and `pandas` provide array and tabular data structures; and machine‑learning functionality comes from `scikit‑learn` and `xgboost`.

In [50]:
# Import python modules
import re
from os import path
import geopandas as gpd
import s3fs
import pandas as pd
import xarray as xr
from shapely.geometry import box, mapping, Point
import rioxarray as riox
import numpy as np
import netCDF4 as nc
import hvplot.xarray
import holoviews as hv
import xvec
import shap
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
import warnings
import rasterio
import hvplot.pandas
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import xgboost as xgb

warnings.filterwarnings('ignore')
hvplot.extension('bokeh')
hv.extension('bokeh')

## Parsing ENVI headers

AVIRIS Level 2A data are distributed as ENVI files consisting of a binary data cube and an ASCII header (`.hdr`). The header stores the metadata, the centre wavelengths for each band, a bad‑band list (BBL) marking bands contaminated by atmospheric absorption, and a nodata value. The helper function `parse_envi_header` reads the header and returns these factors, helping me filter noisy bands.

In [56]:
# Header parser: wavelengths, FWHM, BBL, nodata 
def parse_envi_header(hdr_file: str):
    """Return {'wavelengths': np.ndarray[nm], 'bbl_mask': np.ndarray[bool] or None, 'nodata': float|None}."""
    with open(hdr_file, 'r', encoding='utf-8', errors='ignore') as f:
        txt = f.read()

    def _list(key):
        m = re.search(rf'{key}\s*=\s*\{{([^}}]*)\}}', txt, flags=re.IGNORECASE | re.DOTALL)
        if not m:
            return None
        vals = [v for v in re.split(r'[,\s]+', m.group(1).strip()) if v]
        return np.array([float(v) for v in vals], dtype=float)

    wl = _list('wavelength')
    if wl is None:
        raise ValueError("Header has no 'wavelength' entry.")

    # units → nm
    um = re.search(r'wavelength\s+units\s*=\s*([^\n\r]+)', txt, re.IGNORECASE)
    units = um.group(1).strip().strip('{}').lower() if um else 'nanometers'
    if ('micro' in units) or ('µm' in units) or ('microm' in units):
        wl = wl * 1000.0

    # bbl (1=good, 0=bad) → boolean; guard against inverted conventions
    bbl = _list('bbl')
    bbl_mask = None
    if bbl is not None:
        bbl_mask = (bbl.astype(int) == 1)
        if bbl_mask.mean() < 0.1:
            bbl_mask = ~bbl_mask

    # nodata
    nd = re.search(r'data\s+ignore\s+value\s*=\s*([-\d\.Ee+]+)', txt, re.IGNORECASE)
    nodata = float(nd.group(1)) if nd else None

    return {'wavelengths': wl, 'bbl_mask': bbl_mask, 'nodata': nodata}

## Extracting wavelength metadata

In this cube, I called `parse_envi_header` to obtain the array of band centre wavelengths (in nanometres), the bad‑band mask, and the nodata value. Assigning them to variables upfront makes it straightforward to reference wavelengths and masks throughout the notebook.

In [57]:
# 1. Parse the header for wavelengths, nodata, and BBL
hdr = parse_envi_header(hdr_path)
wavelengths = hdr["wavelengths"]
hdr_bbl     = hdr["bbl_mask"]
hdr_nodata  = hdr["nodata"]

## Loading occurrence points and reprojecting

Occurrence points recorded in the geopackage use a geographic coordinate reference system. To align them with the AVIRIS raster, I read the raster to obtain its CRS and then reproject the `GeoDataFrame` accordingly. Only after this reprojection can we sample reflectance values at the correct pixel locations.

Note: _Occurrence points come as a .txt format. I converted them to geopackage via QGIS._

In [58]:
# 2. Load and reproject your silvertree points
gdf = gpd.read_file(gpkg_path)
with rasterio.open(cube_path) as src:
    raster_crs = src.crs
gdf = gdf.to_crs(raster_crs)

## Computing the flight‑line footprint

Each AVIRIS scene covers a rectangular footprint defined by its bounding coordinates. Here I open the raster and extract the minimum and maximum `x` and `y` coordinates to form a `shapely` box. This footprint is later used to filter occurrence points to those that lie within the scene.

In [59]:
# 3. Find the flightline footprint
with rasterio.open(cube_path) as src:
    xmin, ymin, xmax, ymax = src.bounds
footprint = box(xmin, ymin, xmax, ymax)

## Filtering points within the footprint

After constructing the footprint, I keep only the occurrence points whose geometries fall inside the box. This ensures that the subsequent sampling is restricted to the area covered by the AVIRIS flight line.

In [60]:
# 4. Filter to points inside the footprint
gdf = gdf[gdf.geometry.within(footprint)]
assert len(gdf) > 0, "No silvertree occurrences in this flightline."

## Generating background samples

To train a binary classifier, we need examples of both positive (silvertree) and negative (background) locations. So I buffered each silvertree point by approximately 15 m (three pixels) to avoid sampling background points too close to the target. Then I generated random points within the footprint until I have as many background points as positives. This procedure was to basically create a balanced dataset for model training.

In [61]:
# 5. Generate random background points (same count as positives)
#    We buffer the silvertree points by ~15 m (3 pixels) to avoid overlap
buffered_silvertree = gdf.buffer(15)
background_points = []
rng = np.random.default_rng(seed=42)
while len(background_points) < len(gdf):
    rand_x = rng.uniform(xmin, xmax)
    rand_y = rng.uniform(ymin, ymax)
    pt = Point(rand_x, rand_y)
    if footprint.contains(pt) and not buffered_silvertree.geometry.intersects(pt).any():
        background_points.append(pt)

## Sampling spectral signatures

With positive and negative coordinates prepared, I sampled the reflectance cube at these locations. I stored the spectra in an array called `spectra` and the corresponding labels in a list. The sampling uses `rasterio` to read pixel values and yields a two‑dimensional array of shape (n_samples × n_bands).

In [62]:
# 6. Sample spectra at both silvertree and background points
coords_positive  = [(p.x, p.y) for p in gdf.geometry]
coords_negative  = [(p.x, p.y) for p in background_points]
all_coords       = coords_positive + coords_negative
labels           = [1]*len(coords_positive) + [0]*len(coords_negative)

with rasterio.open(cube_path) as src:
    spectra = np.vstack(list(src.sample(all_coords))).astype("float32")
    nodata  = src.nodata if src.nodata is not None else (hdr_nodata if hdr_nodata is not None else np.nan)
    if np.isfinite(nodata):
        spectra[spectra == nodata] = np.nan

## Cleaning the spectral data

Raw reflectance cubes may contain sentinel values (e.g., −32768) indicating missing data, or physically implausible reflectances (< −0.05 or > 1.5). We replace these values with `NaN`, which allows later functions to ignore them. Removing these values reduces noise and improves model performance.

In [63]:
# 7. Clean the spectra: remove other sentinels and implausible values
for sentinel in (-32768, -32767, -1e20, 1e20):
    spectra[spectra == sentinel] = np.nan
# mask reflectances < −0.05 or > 1.5
spectra[(spectra < -0.05) | (spectra > 1.5)] = np.nan

## Applying a bad‑band mask

Not all spectral bands are equally informative. Atmospheric absorption windows around 1.4 µm and 1.9 µm can introduce noise, so we define a manual bad‑band list that masks these regions. We then combine this manual mask with the header’s BBL, to produce a final `good_bands` mask. Only bands marked as good are retained.

Note: _The previous code of bbl failed to mask bad bands, so I had to manually apply the bad bands from the header metadata and it worked._

In [64]:
# 8. Apply your bad‑band mask (manual + header)
manual_bbl = np.ones_like(wavelengths, dtype=bool)
manual_bbl[197:206] = False
manual_bbl[285:318] = False
good_bands = manual_bbl & (hdr_bbl if hdr_bbl is not None else manual_bbl)
spectra[:, ~good_bands] = np.nan
wavelengths_good = wavelengths[good_bands]

## Selecting valid observations

To ensure that the classifier is trained on reliable data, I dropped samples with more than 10 % missing values across the good bands. For the remaining samples, I optionally filled missing values with bandwise means. The resulting matrix `X` and label array `y` serve as inputs to the classifier.

In [65]:
# 9. Drop any rows with too many missing values (or fill with bandwise means)
mask_valid = np.isfinite(spectra).sum(axis=1) >= int(0.9 * len(wavelengths_good))
X = spectra[mask_valid][:, good_bands]
y = np.array(labels)[mask_valid]
# Simple imputation: fill any NaNs with the bandwise mean
band_means = np.nanmean(X, axis=0)
inds       = np.where(np.isnan(X))
X[inds]    = np.take(band_means, inds[1])

## Splitting into training and test sets

Using `train_test_split` with stratification preserves the proportion of positive and negative samples in both training and test sets. A random seed ensures reproducibility. We hold out 30 % of the data for testing to estimate generalisation performance.

In [66]:
# 10. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

## Hyperparameter tuning with XGBoost

I fit an `XGBClassifier`, optimising learning rate, maximum depth, and number of trees via a grid search. Five‑fold cross‑validation estimates performance for each combination, and the best model is selected based on cross‑validated metrics. Gradient boosting often performs well on high‑dimensional data and can capture non‑linear relationships between reflectance and species identity.

In [67]:
# 11. Hyperparameter tuning with XGBoost
param_grid = {
    "learning_rate": [0.05, 0.1],
    "max_depth": [5, 10],
    "n_estimators": [100, 300],
}
xgb_model  = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method="hist",
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)
grid_search = GridSearchCV(
    xgb_model,
    param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1,
    verbose=0,
)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

## Evaluating the classifier

After training, I use the held‑out test set to predict labels and compute classification metrics via `classification_report`. Precision, recall and F1‑score provide insight into the model’s ability to detect silvertree (positive class) and avoid false positives.

In [68]:
# 12. Evaluate on the test set
y_pred   = best_model.predict(X_test)
report   = classification_report(y_test, y_pred, target_names=["background","silvertree"])
print(report)

              precision    recall  f1-score   support

  background       0.53      0.50      0.52        50
  silvertree       0.64      0.67      0.66        67

    accuracy                           0.60       117
   macro avg       0.59      0.59      0.59       117
weighted avg       0.60      0.60      0.60       117



## Interpreting initial results

Looking at the results from the model prediction, for the fifty background pixels in the held‑out set, the algorithm correctly identified roughly half (about 25 instances) whilst mislabelling the remaining half as silvertrees. For the sixty‑seven silvertree pixels, it retrieved about two‑thirds but assigned the rest to the background. These numbers yield the reported recall of 0.50 for background and 0.67 for silvertrees, and a corresponding precision of 0.53 versus 0.64, which together combine into F₁ scores of 0.52 and 0.66. 

Put differently, when the model declares “silvertree,” there is a one‑in‑three chance it has picked up a background spectrum; when it declares “background,” it is correct barely more than half the time. The overall accuracy of 59%-60% fro macro and weighted averages suggests that the classifier is doing only slightly better than random guessing. In reality though, these values tell me that the conceptual groundwork for class separation is weak. I would imagine the background category likely contains a heterogeneous mix of vegetation, soil and shadow spectra, and the silvertree signature may itself differ with canopy structure and phenology. This is to be expected though since it is iNaturalist points. It is not the exact point where the trees are.

An approach Chat says I can take to improve performance can involve "expanding and diversifying the training dataset, using feature selection or dimensionality‑reduction techniques to identify wavelength regions that most clearly distinguish the species, and exploring alternative learners or spatial contextual filters".

# Sensor accuracy check

This section of code takes an alternative approach inspired by anomaly detection and matched filtering rather than supervised classification. It defines helper functions to create normalised difference vegetation indices (NDVI) and to sample spectral windows, and then constructs training and test sets to evaluate sensor sensitivity. This method acknowledges limitations of the initial classifier and explores whether spectral anomalies could better highlight silvertree crowns. 

Note: _This methods is suggested by ChatGPT._

## Additional imports for matched‑filter detection

For the alternative detector, we import scoring functions such as `roc_auc_score` and `average_precision_score` to evaluate model performance, as well as `savgol_filter` for smoothing spectral curves. These functions support the matched‑filter detection pipeline that follows.

In [69]:
import numpy as np
import geopandas as gpd
import rasterio
from shapely.geometry import box
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve
from scipy.signal import savgol_filter

## Helper functions for spectral analysis

The code defines several functions: `nearest_band_index` locates the nearest wavelength band to a given target; `sample_window_mean` extracts mean spectra within a moving window around a point; `clean_spectrum` removes nodata and implausible values; `extract_features` computes derivative‑based features; and `compute_detector_score` implements the matched‑filter detection.

In [70]:
# --- helpers built for your current objects -----------------------------------

def nearest_band_index(wavelengths, target_nm):
    return int(np.argmin(np.abs(wavelengths - target_nm)))

def sample_window_mean(src, x, y, half=2, good_mask=None, nir_idx=None, red_idx=None, ndvi_min=0.2):
    """Return mean spectrum in a (2*half+1)^2 window around (x,y), masking low-NDVI pixels."""
    row, col = src.index(x, y)
    w = rasterio.windows.Window(col-half, row-half, 2*half+1, 2*half+1)
    # read all bands in the window -> (bands, h, w)
    cube_win = src.read(indexes=None, window=w).astype('float32')
    if cube_win.size == 0:
        return None
    # NDVI mask to de-emphasise soil/shade
    if nir_idx is not None and red_idx is not None:
        nir = cube_win[nir_idx]
        red = cube_win[red_idx]
        ndvi = (nir - red) / (nir + red + 1e-6)
        mask = (ndvi >= ndvi_min) & np.isfinite(ndvi)
    else:
        mask = np.ones_like(cube_win[0], dtype=bool)
    # mean over valid pixels; if none valid, fall back to plain mean
    if mask.sum() == 0:
        spec = np.nanmean(cube_win.reshape(cube_win.shape[0], -1), axis=1)
    else:
        spec = np.nanmean(cube_win[:, mask], axis=1)
    return spec

def brightness_normalise(spec):
    norm = np.linalg.norm(spec[np.isfinite(spec)])
    return spec / (norm + 1e-9)

def first_derivative(spec, window=11, poly=2):
    s = np.copy(spec)
    ok = np.isfinite(s)
    s[~ok] = np.nanmean(s[ok]) if ok.any() else 0.0
    d = savgol_filter(s, window_length=window, polyorder=poly, deriv=1, mode='interp')
    return d

def ace_scores(X, target, C_inv):
    # ACE: ((x^T C^-1 t)^2) / ((t^T C^-1 t)(x^T C^-1 x))
    xt = X @ (C_inv @ target)
    tt = target @ (C_inv @ target)
    xx = np.einsum('ij,ij->i', X @ C_inv, X)
    return (xt**2) / (tt * xx + 1e-12)

## Building training and test sets for matched‑filter detection

We open the raster to determine the nodata value and select the indices of the red and near‑infrared bands for NDVI computation. We then define good bands and compute variables such as the wavelengths of interest. Next, we construct crown proxies by buffering occurrence points by a few pixels, and sample positive spectra using `sample_window_mean`. Each spectrum is cleaned and transformed into features.

In [71]:
# --- build training and test sets of spectra ----------------------------------

# 1) open raster; get nodata; choose NIR/Red band indices for NDVI
with rasterio.open(cube_path) as src:
    rio_nodata = src.nodata
    wavelengths = np.array(cube.coords['band'].values)  # your band coordinate is wavelengths
    nir_idx = nearest_band_index(wavelengths, 840)      # AVIRIS-NG closest to 842 nm
    red_idx = nearest_band_index(wavelengths, 660)

In [72]:
# 2) define good bands: my manual + header BBL
manual_bbl = np.ones_like(wavelengths, dtype=bool)
manual_bbl[197:206] = False
manual_bbl[285:318] = False
good_bands = manual_bbl & (hdr_bbl if hdr_bbl is not None else manual_bbl)
wl_good = wavelengths[good_bands]

In [73]:
# 3) build crown proxies ( Iwill replace this with actual crown measurements when I do field work)
occ = gdf[gdf.geometry.within(box(*rasterio.open(cube_path).bounds))]
occ_buf = occ.buffer(3.0)  # ~5×5 pixels at 4.8 m if half=2 below

In [74]:
# 4) sample positive spectra (window mean + cleaning + features)
pos_specs = []
with rasterio.open(cube_path) as src:
    for geom in occ_buf.centroid:  # centroid sampling; for true polygons, we’d tile
        spec = sample_window_mean(src, geom.x, geom.y, half=2,
                                  nir_idx=nir_idx, red_idx=red_idx, ndvi_min=0.2)
        if spec is None: 
            continue
        # nodata and range cleaning
        if rio_nodata is not None and np.isfinite(rio_nodata):
            spec[spec == rio_nodata] = np.nan
        spec[(spec < -0.05) | (spec > 1.5)] = np.nan
        spec = spec[good_bands]
        if np.isfinite(spec).sum() < 0.9 * spec.size:
            continue
        # modest pre-processing to stabilise detector
        spec = brightness_normalise(spec)
        spec = first_derivative(spec, window=11, poly=2)
        pos_specs.append(spec)
pos_specs = np.array(pos_specs)
assert len(pos_specs) >= 20, "Too few positive spectra; expand buffers or add crowns."

## Sampling background spectra for matched‑filter detection

To create a balanced evaluation, we sample spectra at random locations away from the buffered positive crowns. A spatial index (`STRtree`) facilitates searching for regions sufficiently distant from positives. Spectra are sampled, cleaned and converted to features, forming a set of negative examples.

In [75]:
# 5) sample matched background spectra away from positives
from shapely.strtree import STRtree
tree = STRtree(list(occ_buf.geometry))
footprint = box(*rasterio.open(cube_path).bounds)
rng = np.random.default_rng(42)
neg_specs = []
with rasterio.open(cube_path) as src:
    while len(neg_specs) < len(pos_specs):
        x = rng.uniform(footprint.bounds[0], footprint.bounds[2])
        y = rng.uniform(footprint.bounds[1], footprint.bounds[3])
        p = box(x, y, x, y).centroid
        # reject if near positives
        if tree.query(p.buffer(15)).size > 0: 
            continue
        spec = sample_window_mean(src, x, y, half=2, nir_idx=nir_idx, red_idx=red_idx, ndvi_min=0.2)
        if spec is None: 
            continue
        if rio_nodata is not None and np.isfinite(rio_nodata):
            spec[spec == rio_nodata] = np.nan
        spec[(spec < -0.05) | (spec > 1.5)] = np.nan
        spec = spec[good_bands]
        if np.isfinite(spec).sum() < 0.9 * spec.size:
            continue
        spec = brightness_normalise(spec)
        spec = first_derivative(spec, window=11, poly=2)
        neg_specs.append(spec)
neg_specs = np.array(neg_specs)

## Constructing the matched‑filter detector

We compute a target signature as the median spectrum of positive examples and estimate the background mean and covariance from negatives. The matched‑filter detector then scores each spectrum by its similarity to the target relative to background variability. Higher scores indicate a closer match to the silvertree signature.

In [76]:
# --- fit detector and evaluate ------------------------------------------------

# target signature = median positive spectrum; background covariance from negatives
target = np.nanmedian(pos_specs, axis=0)
mu_b   = np.nanmean(neg_specs, axis=0)
X_b    = np.where(np.isfinite(neg_specs), neg_specs, mu_b)
C_b    = np.cov(X_b.T) + 1e-6 * np.eye(X_b.shape[1])  # small ridge for stability
Cinv   = np.linalg.pinv(C_b)

## Evaluating matched‑filter performance

We split the positive and negative spectra into training and testing subsets and compute scores for each. Receiver operating characteristic (ROC) and precision–recall curves (computed later) can summarise performance. The immediate printout gives an idea of detection efficacy on the held‑out samples.

In [77]:
# scores on held-out sets (simple split here; use spatial CV in practice)
n = len(pos_specs)
split = int(0.7*n)
X_pos_train, X_pos_test = pos_specs[:split], pos_specs[split:]
m = len(neg_specs)
split_n = int(0.7*m)
X_neg_train, X_neg_test = neg_specs[:split_n], neg_specs[split_n:]

## Avoiding information leakage

To avoid overestimating performance, we recompute the target signature using only training positives and the background covariance using only training negatives. We then score both training and testing samples. This practice mirrors common machine‑learning protocol by preventing data leakage from the test set.

In [78]:
# recompute target on training only (no peeking)
target = np.nanmedian(X_pos_train, axis=0)
C_b    = np.cov(np.where(np.isfinite(X_neg_train), X_neg_train, mu_b).T) + 1e-6*np.eye(X_b.shape[1])
Cinv   = np.linalg.pinv(C_b)

scores_pos = ace_scores(X_pos_test, target, Cinv)
scores_neg = ace_scores(X_neg_test, target, Cinv)

y_true  = np.r_[np.ones_like(scores_pos), np.zeros_like(scores_neg)]
y_score = np.r_[scores_pos, scores_neg]

roc_auc = roc_auc_score(y_true, y_score)
ap      = average_precision_score(y_true, y_score)
print(f"ACE ROC-AUC: {roc_auc:.3f}   Average Precision: {ap:.3f}")

ACE ROC-AUC: 0.612   Average Precision: 0.626


## Interpreting matched‑filter results

The Adaptive Cosine Estimator (ACE) works by computing a median target spectrum from the training positives and whitening the test data with a covariance model gathered from the background. The ROC‑AUC of 0.612 suggests that if I randomly pick a silvertree pixel and a background pixel, the detector will assign a higher score to the silvertree only about 61% of the time; this is barely above the fifty percent one expects from random ranking. The average precision of 0.626, which collapses the precision–recall trade‑off into a single summary statistic, also tells us a similar story. The detector’s precision falls off rapidly as recall increases, pointing to a high rate of false alarms. This suggests that the conceptual groundwork of the ACE approach using a single median spectrum as the target and a global covariance for the background does not capture the variability of both silvertrees and the heterogeneous background.

## Extending analysis across multiple scenes

I used the same approach as the above precision test, however, in this approach I increased the diversity of training examples and assessed how detection performance scales with a wider spatial coverage.

In [79]:
# Import modules
import numpy as np
import geopandas as gpd
import rasterio
from shapely.geometry import box, Point
from shapely.strtree import STRtree
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.signal import savgol_filter
from rasterio.windows import Window
import os

In [81]:
# --- Utility functions --------------------------------------------------------

def parse_envi_header(hdr_file):
    """Parse wavelengths, BBL and nodata from an ENVI header."""
    import re
    with open(hdr_file, 'r', encoding='utf-8', errors='ignore') as f:
        txt = f.read()
    def _list(key):
        m = re.search(rf'{key}\s*=\s*\{{([^}}]*)\}}', txt, flags=re.IGNORECASE|re.DOTALL)
        if not m: return None
        vals = [v for v in re.split(r'[,\s]+', m.group(1).strip()) if v]
        return np.array([float(v) for v in vals], dtype=float)
    wl = _list('wavelength')
    units = re.search(r'wavelength\s+units\s*=\s*([^\n\r]+)', txt, re.IGNORECASE)
    units = units.group(1).strip().strip('{}').lower() if units else 'nanometers'
    if 'micro' in units or 'µm' in units:
        wl = wl * 1000.0
    bbl = _list('bbl')
    bbl_mask = None
    if bbl is not None:
        bbl_mask = (bbl.astype(int) == 1)
        if bbl_mask.mean() < 0.1:
            bbl_mask = ~bbl_mask
    nd = re.search(r'data\s+ignore\s+value\s*=\s*([-\d\.Ee+]+)', txt, re.IGNORECASE)
    nodata = float(nd.group(1)) if nd else None
    return {'wavelengths': wl, 'bbl_mask': bbl_mask, 'nodata': nodata}

def nearest_band_index(wavelengths, target_nm):
    return int(np.argmin(np.abs(wavelengths - target_nm)))

def sample_window_mean(src, x, y, half=2, nir_idx=None, red_idx=None, ndvi_min=0.2):
    """Return mean spectrum in a (2*half+1)^2 window around (x,y), masking low-NDVI pixels."""
    row, col = src.index(x, y)
    w = Window(col - half, row - half, 2*half+1, 2*half+1)
    cube_win = src.read(indexes=None, window=w).astype('float32')  # shape: bands × h × w
    if cube_win.size == 0:
        return None
    h, w_ = cube_win.shape[1], cube_win.shape[2]
    # NDVI mask to downweight soil/shade
    if nir_idx is not None and red_idx is not None:
        nir = cube_win[nir_idx]
        red = cube_win[red_idx]
        ndvi = (nir - red) / (nir + red + 1e-9)
        mask = (ndvi >= ndvi_min) & np.isfinite(ndvi)
    else:
        mask = np.ones_like(cube_win[0], dtype=bool)
    # mean across valid pixels
    if mask.sum() == 0:
        spec = np.nanmean(cube_win.reshape(cube_win.shape[0], -1), axis=1)
    else:
        spec = np.nanmean(cube_win[:, mask], axis=1)
    return spec

def brightness_normalise(spec):
    norm = np.linalg.norm(spec[np.isfinite(spec)])
    return spec / (norm + 1e-9)

def first_derivative(spec, window=11, poly=2):
    s = np.copy(spec)
    ok = np.isfinite(s)
    s[~ok] = np.nanmean(s[ok]) if ok.any() else 0.0
    d = savgol_filter(s, window_length=window, polyorder=poly, deriv=1, mode='interp')
    return d

def ace_scores(X, target, C_inv):
    """Compute ACE scores for each row in X given target and inverse covariance."""
    xt = X @ (C_inv @ target)
    tt = target @ (C_inv @ target)
    xx = np.einsum('ij,ij->i', X @ C_inv, X)
    return (xt**2) / (tt * xx + 1e-12)

## Specifying multiple AVIRIS scenes

I added a list of AVIRIS Level 2A cubes covering different flight lines. These scenes will be processed sequentially in a loop, allowing the detector to learn from a broader range of conditions and to test whether spectra from different scenes share a consistent silvertree signature.

In [88]:
# --- Paths to your data -------------------------------------------------------

cube_paths = [
    "data/ALL/ang20231126t084422_000_L2A_OE_main_27577724_RFL_ORT",
    "data/ALL/ang20231126t084422_001_L2A_OE_main_27577724_RFL_ORT",
    "data/ALL/ang20231126t084422_002_L2A_OE_main_27577724_RFL_ORT",
    "data/ALL/ang20231126t084422_003_L2A_OE_main_27577724_RFL_ORT",
    "data/ALL/ang20231126t084422_004_L2A_OE_main_27577724_RFL_ORT",
    "data/ALL/ang20231126t084422_005_L2A_OE_main_27577724_RFL_ORT",
    "data/ALL/ang20231126t084422_006_L2A_OE_main_27577724_RFL_ORT",
    "data/ALL/ang20231126t084422_007_L2A_OE_main_27577724_RFL_ORT",
]
gpkg_path  = "data/silvertree/occurrence.gpkg"
buffer_m   = 3.0  # radius (m) to create small buffers for positive samples
random_seed = 42

In [82]:
# --- Load occurrence points once ---------------------------------------------

gdf_global = gpd.read_file(gpkg_path)

## Initialising containers for combined spectra

I created empty lists `pos_specs_all` and `neg_specs_all` to collect spectra across scenes. We also set placeholders for `global_wavelengths` and `global_bbl` to ensure consistency across cubes.

In [83]:
# --- Containers for combined spectra -----------------------------------------

pos_specs_all = []
neg_specs_all = []
global_wavelengths = None
global_bbl = None

## Looping over AVIRIS cubes to extract spectra

Within a `for` loop, I parse the header of each cube, identify wavelengths and BBL masks, and sample positive and negative spectra following similar procedures to the single‑scene case I used above. I ensured that the global wavelength array and bad‑band mask were consistent across scenes, reproject occurrence points to the cube’s CRS, and accumulate cleaned spectra.

In [89]:
# --- Loop over cubes to extract spectra ---------------------------------------

for cube_path in cube_paths:
    hdr = parse_envi_header(cube_path + ".hdr")
    wl  = hdr['wavelengths']
    bbl = hdr['bbl_mask']
    nd  = hdr['nodata']

    # Use each cube’s own BBL to define its "good bands"
    if bbl is None:
        print(f"{cube_path}: No BBL found; skipping")
        continue
    # Determine "good bands" by this cube’s BBL alone
    good_bands_cube = bbl
    # Keep track of global wavelengths and intersection of BBLs
    if global_wavelengths is None:
        global_wavelengths = wl.copy()
        global_bbl = bbl.copy()
    else:
        if not np.allclose(wl, global_wavelengths):
            print(f"{cube_path}: Wavelengths differ from previous cubes; skipping")
            continue
        global_bbl = global_bbl & bbl

    # Open cube and reproject points
    with rasterio.open(cube_path) as src:
        raster_crs = src.crs
        gdf = gdf_global.to_crs(raster_crs)
        xmin, ymin, xmax, ymax = src.bounds
        foot = box(xmin, ymin, xmax, ymax)
        gdf = gdf[gdf.geometry.within(foot)]
        if len(gdf) == 0:
            print(f"{cube_path}: No silvertrees in this cube")
            continue

        # Sample positive spectra
        pos_cube = []
        for geom in gdf.buffer(buffer_m).centroid:
            spec = sample_window_mean(
                src, geom.x, geom.y, half=2,
                nir_idx=nearest_band_index(wl, 840),
                red_idx=nearest_band_index(wl, 660),
                ndvi_min=0.2,
            )
            if spec is None: continue
            # Remove nodata and outliers
            if nd is not None and np.isfinite(nd):
                spec[spec == nd] = np.nan
            spec[(spec < -0.05) | (spec > 1.5)] = np.nan
            spec = spec[good_bands_cube]
            if np.isfinite(spec).sum() < 0.9 * spec.size:
                continue
            # Preprocess
            spec = brightness_normalise(spec)
            spec = first_derivative(spec, window=11, poly=2)
            pos_cube.append(spec)
        pos_specs_all.extend(pos_cube)

        # Build background sampler
        tree = STRtree(list(gdf.buffer(10.0).geometry))
        rng = np.random.default_rng(seed=random_seed)
        neg_cube = []
        while len(neg_cube) < len(pos_cube):
            x = rng.uniform(xmin, xmax)
            y = rng.uniform(ymin, ymax)
            pt = Point(x, y)
            if not foot.contains(pt) or tree.query(pt).size > 0:
                continue
            spec = sample_window_mean(
                src, x, y, half=2,
                nir_idx=nearest_band_index(wl, 840),
                red_idx=nearest_band_index(wl, 660),
                ndvi_min=0.2,
            )
            if spec is None: continue
            if nd is not None and np.isfinite(nd):
                spec[spec == nd] = np.nan
            spec[(spec < -0.05) | (spec > 1.5)] = np.nan
            spec = spec[good_bands_cube]
            if np.isfinite(spec).sum() < 0.9 * spec.size:
                continue
            spec = brightness_normalise(spec)
            spec = first_derivative(spec, window=11, poly=2)
            neg_cube.append(spec)
        neg_specs_all.extend(neg_cube)

print(f"Collected {len(pos_specs_all)} positive spectra and {len(neg_specs_all)} negative spectra")

data/ALL/ang20231126t084422_005_L2A_OE_main_27577724_RFL_ORT: No silvertrees in this cube
data/ALL/ang20231126t084422_006_L2A_OE_main_27577724_RFL_ORT: No silvertrees in this cube
Collected 297 positive spectra and 297 negative spectra


## Computing the combined detector

After processing all cubes, I stacked the positive and negative spectra and computed a target signature and background covariance across the combined dataset. If no spectra were extracted, an error was raised. The matched‑filter approach is then applied to the aggregated data.

In [90]:
# --- Compute combined detector ------------------------------------------------

pos_specs_all = np.array(pos_specs_all)
neg_specs_all = np.array(neg_specs_all)
if len(pos_specs_all) == 0 or len(neg_specs_all) == 0:
    raise RuntimeError("No spectra collected; adjust buffer or file paths")

# Global good bands: intersection across all cubes
good_bands_global = global_bbl
wl_good           = global_wavelengths[good_bands_global]

# Keep only those bands in all spectra (they already are trimmed to each cube's bbl, but we align them)
# In this simplified approach, we assume the trimming by each cube's BBL gives the same number/order of bands.
# If not, you’d need to resample or pad to match across cubes.
X_pos = pos_specs_all
X_neg = neg_specs_all

# Target signature and background covariance
target = np.nanmedian(X_pos, axis=0)
mu_b   = np.nanmean(X_neg, axis=0)
X_neg  = np.where(np.isfinite(X_neg), X_neg, mu_b)
C_b    = np.cov(X_neg.T) + 1e-6 * np.eye(X_neg.shape[1])
C_inv  = np.linalg.pinv(C_b)

# Evaluate on held-out split
split_pos = int(0.7 * len(X_pos))
split_neg = int(0.7 * len(X_neg))
scores_pos = ace_scores(X_pos[split_pos:], target, C_inv)
scores_neg = ace_scores(X_neg[split_neg:], target, C_inv)
y_true  = np.r_[np.ones_like(scores_pos), np.zeros_like(scores_neg)]
y_score = np.r_[scores_pos, scores_neg]
roc_auc = roc_auc_score(y_true, y_score)
ap      = average_precision_score(y_true, y_score)
print(f"Combined ACE ROC‑AUC: {roc_auc:.3f}   Average Precision: {ap:.3f}")

Combined ACE ROC‑AUC: 0.698   Average Precision: 0.638


## Reflecting on multi‑scene results

The author concludes that combining multiple cubes modestly improves detection performance, pushing precision towards 70 %. Nevertheless, more extensive fieldwork and additional data are recommended to validate and refine the detector. The commentary underscores the iterative nature of remote sensing model development.

By pooling spectra from multiple cubes and recomputing both the median target and the background covariance across this enlarged corpus, I managed to effectively broadened the groundwork on which the Adaptive Cosine Estimator operates, and it can be seen with the new ACE ROC-AUC. The ACE ROC-AUC slightly increased from 0.612 to 0.698, which means that a randomly chosen silvertree pixel now outranks a randomly chosen background pixel nearly seventy percent of the time, which is a substantial increase from the 61% I got when using a single cube. The average precision of 0.638 likewise increases, although not as much, which tells me that there is a slightly improved balance between true detections and false alarms across the full range of thresholds. So broadening the training data has laid the foundation for a more representative target signature and a more reliable background model, thereby sharpening the detector’s ability to distinguish signal from noise.

The fact that the area under the ROC curve remains below 0.7 still indicates that the conceptual groundwork still falls short of capturing the desired outcomes. Spectral variability within silvertrees and heterogeneity in the negative class continue to limit discrimination. What I can try to do is construct multiple target signatures to reflect, and see the outcome.

ALSO, I STILL FEEL AS THOUGH I TESTING THE ACCURACY OF THE MODELS INSTEAD OF THE SENSOR. THE MAIN OBJECTIVE IS TO KNOW WHETHER OR NOT THE SENSOR IS ACCURATE. I WILL ADD MORE SPECIES FOUND IN RHODES MEMORIAL TO SEE HOW THE RESULTS CHANGE.